In [17]:
# basics
import intake
import xarray as xr
import dask  # memory-efficient parallel computation and delayed execution (lazy evaluation).
import subprocess as sp

%run gem_helpers.ipynb
uid = getpass.getuser()
image_path = make_tempdir("intake_demo_plots")
data_cache_path = make_tempdir("intake_demo_data")

# this function is also part of the gem helpers
def get_list_from_cat(catalog, column):
    """A helper function for getting the contents of a column in an intake catalog.
    Call with the catalog to be inspected and the column of interest."""
    return sorted(catalog.unique(column)[column]["values"])

catalog_file = "/work/ka1081/Catalogs/dyamond-nextgems.json"
cat = intake.open_esm_datastore(catalog_file)

In [18]:
# get_from_cat is a function in gem_helpers for overview
get_from_cat(cat.search(simulation_id=["ngc2013"], variable_id="to"), "frequency")

,frequency
0,12day
1,1day
2,1hour
3,1month
4,3hour


In [83]:
#var = "tas"
#var = "sfcwind" #"hfls"

var="to"
simulation="ngc2013"
freq="1month"
hits = cat.search(simulation_id=simulation, frequency=freq,variable_id=var)

In [84]:
get_from_cat(hits,"variable_id")

,variable_id
0,"(to, so, u, v, w, tke, A_veloc_v, A_tracer_v_to, heat_content_liquid_water, vort)"


In [85]:
dataset_dict = hits.to_dataset_dict(cdf_kwargs={"chunks": {"time": 1}})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institution_id.source_id.experiment_id.simulation_id.realm.frequency.time_reduction.grid_label.level_type'


In [124]:
# to plot, we need to associate a grid file with the data. The easiest way is to get the info from the grid_file_uri attribute in the data.
data_dict = {}
for name, dataset in dataset_dict.items():
    print(name)
    grid_file_path = "/pool/data/ICON" + dataset.grid_file_uri.split(".de")[1]
    grid_data = xr.open_dataset(grid_file_path).rename(
        cell="ncells"
    )  # the dimension has different names in the grid file and in the output.
    data = xr.merge((dataset, grid_data))
    fix_time_axis(data)
    data_dict[name] = data

nextGEMS.MPI-M.ICON-ESM.nextgems_cycle2.ngc2013.oce.1month.mean.gn.ml


In [126]:
for name, data in data_dict.items():
    pass
data[var]

<xarray.DataArray 'clon' (ncells: 3729001)>
[3729001 values with dtype=float64]
Coordinates:
    clon     (ncells) float64 ...
    clat     (ncells) float64 ...
Dimensions without coordinates: ncells
Attributes:
    long_name:      center longitude
    units:          radian
    standard_name:  grid_longitude
    bounds:         clon_vertices

In [131]:
to_sel=data.to.isel(time=1).isel(depth=0)

In [141]:
outfile='/work/bb1153/b381639/ICON/SST_landseamask_'+simulation+'.nc'
to_sel.to_netcdf(outfile,engine='scipy')

In [114]:
#xarray dataset
dataset = dataset_dict[keys[0]]
to_sel=dataset.isel(time=1)

In [118]:
outVar=to_surface=to_sel.isel(depth=0).to

In [119]:
outfile='/work/bb1153/b381639/ICON/SST_landseamask_'+simulation+'.nc'

In [120]:
outfile

'/work/bb1153/b381639/ICON/SST_landseamask_ngc2013.nc'

In [123]:
outVar.to_netcdf(outfile,engine='scipy')

In [48]:
simulation_id = "ngc2013"

catalog=hits.search(simulation_id=simulation_id,frequency=['3hour'])

file_cat=sorted(catalog.unique("uri")["uri"]["values"])


In [51]:
outfile_dict = {}
for files in file_cat:
    fileS=files.split(',')
    outfile = f"/work/bb1153/b381639/ICON/{var}_monstd_{simulation_id}.nc"
    if not os.access(outfile, os.R_OK):
        query = (
            [
                "cdo",
                "-P",
                "8",
                "-monstd",
                f"-select,name={var}",
                "[",
            ]
            + fileS #[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200120T000000-20200131T235830/ngc2013_atm_2d_3h_mean_20200120T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200120T000000-20200131T235830/ngc2013_atm_2d_3h_mean_20200201T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200201T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200301T000000Z.nc',

cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200120T000000-20200131T235830/ngc2013_atm_2d_3h_mean_20200120T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200120T000000-20200131T235830/ngc2013_atm_2d_3h_mean_20200201T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200201T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200301T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200401T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/wor

cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200601T000000-20200831T235830/ngc2013_atm_2d_3h_mean_20200801T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200601T000000-20200831T235830/ngc2013_atm_2d_3h_mean_20200901T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200901T000000-20201130T235830/ngc2013_atm_2d_3h_mean_20200901T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200901T000000-20201130T235830/ngc2013_atm_2

cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200601T000000-20200831T235830/ngc2013_atm_2d_3h_mean_20200801T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200601T000000-20200831T235830/ngc2013_atm_2d_3h_mean_20200901T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200901T000000-20201130T235830/ngc2013_atm_2d_3h_mean_20200901T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200901T000000-20201130T235830/ngc2013_atm_2d_3h_mean_20201001T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200901T000000-20201130T235830/ngc2013_atm_2d_3h_mean_20201101T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/wor

cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20201201T000000-20210228T235830/ngc2013_atm_2d_3h_mean_20210301T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20210301T000000-20210531T235830/ngc2013_atm_2d_3h_mean_20210301T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20210301T000000-20210531T235830/ngc2013_atm_2d_3h_mean_20210401T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20210301T000000-20210531T235830/ngc2013_atm_2

...

cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20410401T000000-20410430T235845/ngc2013_atm_2d_3h_mean_20410401T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20410401T000000-20410430T235845/ngc2013_atm_2d_3h_mean_20410501T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20410501T000000-20410531T235845/ngc2013_atm_2d_3h_mean_20410501T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20410501T000000-20410531T235845/ngc2013_atm_2

cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20410401T000000-20410430T235845/ngc2013_atm_2d_3h_mean_20410401T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20410401T000000-20410430T235845/ngc2013_atm_2d_3h_mean_20410501T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20410501T000000-20410531T235845/ngc2013_atm_2d_3h_mean_20410501T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20410501T000000-20410531T235845/ngc2013_atm_2d_3h_mean_20410601T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20410601T000000-20410630T235845/ngc2013_atm_2d_3h_mean_20410601T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/wor

cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20410801T000000-20410831T235845/ngc2013_atm_2d_3h_mean_20410901T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20410901T000000-20410930T235845/ngc2013_atm_2d_3h_mean_20410901T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20410901T000000-20410930T235845/ngc2013_atm_2d_3h_mean_20411001T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20411001T000000-20411031T235845/ngc2013_atm_2

cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20410801T000000-20410831T235845/ngc2013_atm_2d_3h_mean_20410901T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20410901T000000-20410930T235845/ngc2013_atm_2d_3h_mean_20410901T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20410901T000000-20410930T235845/ngc2013_atm_2d_3h_mean_20411001T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20411001T000000-20411031T235845/ngc2013_atm_2d_3h_mean_20411001T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20411001T000000-20411031T235845/ngc2013_atm_2d_3h_mean_20411101T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/wor

KeyboardInterrupt: 

In [50]:
outfile_dict = {}
for files in file_cat:
    fileS=files.split(',')
    outfile = f"/work/bb1153/b381639/ICON/{var}_monstd_{simulation_id}.nc"
    if not os.access(outfile, os.R_OK):
        query = (
            [
                "cdo",
                "-P",
                "8",
                "-monstd",
                f"-select,name={var}",
                "[",
            ]
            + fileS[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200120T000000-20200131T235830/ngc2013_atm_2d_3h_mean_20200120T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']


cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200120T000000-20200131T235830/ngc2013_atm_2d_3h_mean_20200120T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200120T000000-20200131T235830/ngc2013_atm_2d_3h_mean_20200201T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200201T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200301T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200401T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/wor

cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200120T000000-20200131T235830/ngc2013_atm_2d_3h_mean_20200201T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200201T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200301T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2

cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200601T000000-20200831T235830/ngc2013_atm_2d_3h_mean_20200601T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200601T000000-20200831T235830/ngc2013_atm_2d_3h_mean_20200701T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200601T000000-20200831T235830/ngc2013_atm_2d_3h_mean_20200801T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200601T000000-20200831T235830/ngc2013_atm_2d_3h_mean_20200901T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20200901T000000-20201130T235830/ngc2013_atm_2d_3h_mean_20200901T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/wor

['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200601T000000-20200831T235830/ngc2013_atm_2d_3h_mean_20200701T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200601T000000-20200831T235830/ngc2013_atm_2d_3h_mean_20200801T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200601T000000-20200831T235830/ngc2013_atm_2d_3h_mean_20200901T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200901T000000-20201130T235830/ngc2013_atm_2d_3h_mean_20200901T000000Z.nc',

...

cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20490301T000000-20490331T235845/ngc2013_atm_2d_3h_mean_20490301T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20490301T000000-20490331T235845/ngc2013_atm_2d_3h_mean_20490401T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20490401T000000-20490430T235845/ngc2013_atm_2d_3h_mean_20490401T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20490401T000000-20490430T235845/ngc2013_atm_2d_3h_mean_20490501T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20490501T000000-20490531T235845/ngc2013_atm_2d_3h_mean_20490501T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/wor

cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20490501T000000-20490531T235845/ngc2013_atm_2d_3h_mean_20490501T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20490501T000000-20490531T235845/ngc2013_atm_2d_3h_mean_20490601T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20490601T000000-20490630T235845/ngc2013_atm_2d_3h_mean_20490601T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20490601T000000-20490630T235845/ngc2013_atm_2

cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20490701T000000-20490731T235845/ngc2013_atm_2d_3h_mean_20490701T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20490701T000000-20490731T235845/ngc2013_atm_2d_3h_mean_20490801T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20490801T000000-20490831T235845/ngc2013_atm_2d_3h_mean_20490801T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20490801T000000-20490831T235845/ngc2013_atm_2d_3h_mean_20490901T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20490901T000000-20490930T235845/ngc2013_atm_2d_3h_mean_20490901T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/wor

cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20490901T000000-20490930T235845/ngc2013_atm_2d_3h_mean_20490901T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20490901T000000-20490930T235845/ngc2013_atm_2d_3h_mean_20491001T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20491001T000000-20491031T235845/ngc2013_atm_2d_3h_mean_20491001T000000Z.nc', ']', '/work/bb1153/b381639/ICON/sfcwind_monstd_ngc2013.nc']
cdo(1) select: Process started
['cdo', '-P', '8', '-monstd', '-select,name=sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20491001T000000-20491031T235845/ngc2013_atm_2

cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20491001T000000-20491031T235845/ngc2013_atm_2d_3h_mean_20491101T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20491101T000000-20491130T235845/ngc2013_atm_2d_3h_mean_20491101T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20491101T000000-20491130T235845/ngc2013_atm_2d_3h_mean_20491201T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20491201T000000-20491231T235845/ngc2013_atm_2d_3h_mean_20491201T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/work/bm1235/k203123/experiments/ngc2013/run_20491201T000000-20491231T235845/ngc2013_atm_2d_3h_mean_20500101T000000Z.nc<
               No such file or directory
cdo(1) select: Open failed on >/wor

In [44]:
for files in file_cat:
    fileS=files#.split(',')
outfile = f"/work/bb1153/b381639/ICON/{var}_{fileS}.nc"

In [45]:
outfile

'/work/bb1153/b381639/ICON/sfcwind_/work/bm1235/k203123/experiments/ngc2013/run_20500101T000000-20500131T235845/ngc2013_atm_2d_3h_mean_20500201T000000Z.nc.nc'

In [20]:
for files in file_cat:
    fileS=files.split(',')
    outfile = f"/work/bb1153/b381639/ICON/{var}_daymean_{simulation_id}.nc"
    if True:
        query = (
            [
                "cdo",
                "-P",
                "4",
                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + fileS[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

['/work/bm1235/k203123/experiments/ngc2013/run_20500101T000000-20500131T235845/ngc2013_atm_2d_3h_mean_20500201T000000Z.nc']

In [15]:



    if True:
        query = (
            [
                "cdo",
                "-P",
                "4",
#                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + files[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
#    outfile_dict[simulation_id] = outfile

TypeError: unhashable type: 'slice'

In [24]:
for files in file_cat:
    fileS=files.split(',')

In [25]:
print(fileS)

['/work/bm1235/k203123/experiments/ngc2012/run_20271101T000000-20280229T235830/ngc2012_atm_2d_3h_mean_20280201T000000Z.nc']


In [26]:
for files in file_cat:
    fileS=files.split(',')
    outfile = f"{data_cache_path}/{var}_daymean_{simulation_id}.nc"
    if True:
        query = (
            [
                "cdo",
                "-P",
                "4",
                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + fileS[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

['cdo', '-P', '4', '-remapcon,r360x180', '-daymean', '-sellonlatbox,300,310,5,17', '-cat', '-apply,-selname,sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2012/run_20200120T000000-20200131T235830/ngc2012_atm_2d_3h_mean_20200120T000000Z.nc', ']', '/scratch/b/b381639/intake_demo_data/sfcwind_daymean_ngc2012.nc']
cdo(1) daymean: Process started
cdo(2) sellonlatbox: Process started
cdo(3) cat: Process started
cdo(4) selname: Process started
cdo    remapcon: YAC first order conservative weights from unstructured (14701) to lonlat (360x180) grid
cdo(4) selname: Processed 503316480 values from 26 variables over 96 timesteps.
cdo(3) cat: Processed 503316480 values from 1 variable over 96 timesteps.
cdo(2) sellonlatbox: Processed 503316480 values from 1 variable over 96 timesteps.
cdo(1) daymean: Processed 1411296 values from 1 variable over 107 timesteps.
cdo    remapcon: Processed 176412 values from 1 variable over 12 timesteps [10.88s 2043MB].


NameError: name 'outfile_dict' is not defined

In [16]:
        query = (
            [
                "cdo",
                "-P",
                "4",
                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + files[:10]
            + ["]", outfile]
        )

NameError: name 'files' is not defined

In [71]:
outfile_dict = {}
for simulation_id, files in file_cat.items():
    outfile = f"{data_cache_path}/{var}_daymean_{simulation_id}.nc"
    if True: # not os.access(outfile, os.R_OK):
        query = (
            [
                "cdo",
                "-P",
                "4",
                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + files[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

'/scratch/b/b381639/intake_demo_data/sfcwind_daymean_1.nc'

In [ ]:
data_dict = {}
for name, dataset in dataset_dict.items():
    print(name)
    grid_file_path = "/pool/data/ICON" + dataset.grid_file_uri.split(".de")[1]
    grid_data = xr.open_dataset(grid_file_path).rename(
        cell="ncells"
    )  # the dimension has different names in the grid file and in the output.
    data = xr.merge((dataset, grid_data))
    fix_time_axis(data)
    data_dict[name] = data